# Solving the equation
## $-\nabla \cdot ( q(u) \nabla u ) = f$

In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import sympy as sym
import numpy as np

In [ ]:
def q(u):
    "Return nonlinear coefficient"
    return 1 + u**2

In [ ]:
# Use SymPy to compute f from the manufactured solution u
x, y = sym.symbols('x[0], x[1]')
u = 1 + x + 2*y
f = - sym.diff(q(u)*sym.diff(u, x), x) - sym.diff(q(u)*sym.diff(u, y), y)
f = sym.simplify(f)
u_code = sym.printing.ccode(u)
f_code = sym.printing.ccode(f)
print('u =', u_code)
print('f =', f_code)

In [ ]:
# Create mesh and define function space
mesh = UnitSquareMesh(16, 16)
V = FunctionSpace(mesh, 'P', 1)
u = Function(V)  # Note: not TrialFunction!
v = TestFunction(V)

In [ ]:
def boundary(x, on_boundary):
    return on_boundary

# Define boundary condition
u_D = Expression(u_code, degree=2)
bc = DirichletBC(V, u_D, boundary)

In [ ]:
# Define variational problem
f = Expression(f_code, degree=2)
F = q(u)*dot(grad(u), grad(v))*dx - f*v*dx
# Compute solution
solve(F == 0, u, bc)

In [ ]:
# Plot solution
p = plot(u)
p.set_cmap("seismic")
plt.title("Poisson solution")
plt.colorbar(p)
plot(mesh)
plt.savefig('u.png')
plt.show()


In [ ]:
# Compute maximum error at vertices. This computation illustrates
# an alternative to using compute_vertex_values as in poisson.py.
u_e = interpolate(u_D, V)
error_max = np.abs(u_e.vector().get_local() - u.vector().get_local()).max()
print('error_max = ', error_max)